## Trade Agreements Database

In [1]:
import graph_tool.all as gt
import graph_tool.stats as gt_stats
import matplotlib
import matplotlib.colors as colors
from matplotlib.font_manager import FontProperties
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.pyplot import clf, xlabel, ylabel, imshow, colorbar, savefig
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import os
import pyreadr

In [2]:
os.chdir('../..')

pathpta = os.path.abspath(f'../02 Data/PTA Database')
pathcepii = os.path.abspath(f'../02 Data/CEPII')
pathmrio = os.path.abspath(f'../../MRIO')
pathindicators = os.path.abspath(f'../../GVC 2023/Indicators')

### I. Bilateral agreements

Construct PTA database with depth measures

In [3]:
pta_raw = pd.read_excel(f'{pathpta}/01 Horizontal Depth_bilateral.xlsx')

In [4]:
pta = pta_raw

# list of columns 
wto_plus = pta.filter(regex=r'^wto_plus_.*_le$').columns.tolist()
wto_x =  pta.filter(regex=r'^wto_X_.*_le$').columns.tolist()
total = wto_plus + wto_x
core = wto_plus + ["wto_X_competitionpolicy_le", 
                   "wto_X_ipr_le", 
                   "wto_X_investment_le", 
                   "wto_X_movementofcapital_le"]
other_econ = ['wto_X_consumerprotection_le', 'wto_X_dataprotection_le', 'wto_X_agriculture_le', 'wto_X_approxlegis_le',
              'wto_X_civilprotection_le', 'wto_X_educationandtraining_le', 'wto_X_energy_le', 'wto_X_financialassistance_le',
              'wto_X_industrialcooperation_le', 'wto_X_mining_le', 'wto_X_nuclearsafety_le', 'wto_X_publicadministration_le',
              'wto_X_regionalcooperation_le', 'wto_X_sme_le', 'wto_X_statistics_le', 'wto_X_taxation_le']

depth_list = {'wto_plus': wto_plus, 'wto_x': wto_x, 'total': total, 'core': core, 'econ':other_econ}

# add depth columns
for depth in depth_list:
    pta['depth_' + str(depth)] = pta.filter(items=depth_list[depth], axis=1).clip(lower=0, upper=1).abs().sum(axis=1)

In [5]:
pta[['depth_total', 'depth_wto_plus', 'depth_wto_x', 'depth_core', 'depth_econ']].describe().applymap('{:,.0f}'.format)

,depth_total,depth_wto_plus,depth_wto_x,depth_core,depth_econ
count,"159,118","159,118","159,118","159,118","159,118"
mean,23,11,12,13,5
std,13,4,9,5,5
min,0,0,0,0,0
25%,11,8,3,10,0
50%,28,12,16,15,5
75%,36,14,22,17,9
max,44,14,32,18,14


Extend PTA database to 2022

In [6]:
# trade agreements active from 2015 onwards
inactive = ['Turkey-EFTA', 'EAEC', 'Nicaragua - Chinese Taipei', 'NAFTA', 'Turkey - Jordan']
pta_2015 = pta[(pta['year'] == 2015) & (~pta['agreement'].isin(inactive))]

# years to be added 
add_years = list(range(2016, 2023))

# copy columns
pta_supplement = pd.DataFrame(columns=pta_2015.columns)

# additional rows 2016-2022
for year in add_years:
    row = pta_2015.copy()
    row['year'] = year
    pta_supplement = pd.concat([pta_supplement, row], ignore_index=True)

In [7]:
pta_full = pd.concat([pta, pta_supplement], ignore_index=True)

### II. Borin Mancini decomposition

In [17]:
# BM decomposition 2000, 2007-2022
ta_72 = pd.read_parquet(f'{pathindicators}/Trade Accounting/ta.parquet')
ta_62 = pd.read_parquet(f'{pathindicators}/Trade Accounting/ta62.parquet')

ta_62 = ta_62[ta_62['t'].astype(int) < 2017] # remove 2017 onwards
ta_62.loc[ta_62['s'] == 63, 's'], ta_62.loc[ta_62['r'] == 63, 'r'] = 73, 73 # change RoW code

ta = pd.concat([ta_62, ta_72], ignore_index=True) # combine

In [18]:
# calculate linkage measures
ta['REX'] = ta['REX1'] + ta['REX2'] + ta['REX3']
ta['REF'] = ta['REF1'] + ta['REF2']

ta['DVA'] = ta['DAVAX1'] + ta['DAVAX2'] + ta['REX'] + ta['REF'] # total DVA
ta['DVA_int'] = ta['DAVAX2'] + ta['REX'] + ta['REF'] # DVA intermediates 
ta['DVA_intrex'] = ta['REX'] + ta['REF'] # DVA re-exported

ta['PDC'] = ta['PDC1'] + ta['PDC2'] # PDC

In [19]:
# standardize country names
countries = pd.read_excel(f'{pathmrio}/countries.xlsx').dropna(subset=['mrio'])
countries = countries[['iso_a3', 'name_adb', 'region_wb', 'mrio']]
countries['mrio'] = countries['mrio'].astype(int)

countries_map, iso_map, region_map = dict(zip(countries['mrio'], countries['name_adb'])), dict(zip(countries['mrio'], countries['iso_a3'])), dict(zip(countries['mrio'], countries['region_wb']))

ta['source_iso'], ta['source_name'], ta['receive_iso'], ta['receive_name']  = ta['s'].map(iso_map), ta['s'].map(countries_map), ta['r'].map(iso_map), ta['r'].map(countries_map)
ta['source_region'], ta['receive_region'] = ta['s'].map(region_map), ta['r'].map(region_map)

In [20]:
decomp = ta[['t', 'source_iso', 'source_name', 'source_region', 'receive_iso', 'receive_name', 'receive_region', 'Exports', 'DVA', 'DVA_int', 'DVA_intrex', 'FVA', 'PDC']]
decomp.loc[:, 't'] = decomp['t'].astype(int)

### III. Gravity model variables

In [21]:
gravity = pd.read_csv(f'{pathcepii}/Gravity_V202211.csv')

/var/folders/rd/n9w0gpv53y72x5k9wk3hp63w0000gn/T/ipykernel_555/2714760110.py:1: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  gravity = pd.read_csv(f'{pathcepii}/Gravity_V202211.csv')


### IV. Merged dataset

In [22]:
decomp = decomp[(decomp['source_iso'] != 'ROW') & (decomp['receive_iso'] != 'ROW')] # remove RoW

decomp['partners'] = decomp.apply(lambda row: tuple(sorted([row['source_iso'], row['receive_iso']])), axis=1)
pta_full['partners'] = pta_full.apply(lambda row: tuple(sorted([row['iso1'], row['iso2']])), axis=1)

In [23]:
merged = decomp.merge(pta_full, left_on=['partners', 't'], right_on=['partners', 'year'], how='left', indicator=True) # merge decomp and pta dfs 
merged['_merge'].unique() # check values 

['left_only', 'both']
Categories (3, object): ['left_only', 'right_only', 'both']

In [24]:
merged = merged.merge(gravity, left_on=['source_iso', 'receive_iso', 't'], right_on=['iso3_o', 'iso3_d', 'year'], how='left') # merge with gravity df
merged.to_parquet('data/01_merged data full.parquet')

### V. Kazakhstan dataset

In [25]:
kaz = merged[(merged['source_iso'] == 'KAZ')]
kaz.to_parquet('data/02_merged data kazakhstan.parquet')